In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 19 17:08:19 2024

@author: Rodrigo Meira
"""
from libs.eos_database import *
from libs.compressor_class import *
from libs.compression import *
from libs.gc_eos_soave import *
from libs.viscosity import *
from libs.plenum_system import *
from libs.simulation import *
from libs.model import *

from casadi import *
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import torch
from scipy.stats import qmc
from sklearn.model_selection import train_test_split

In [2]:
list_names = ["CH4", "C2H6", "C3H8", "iC4H10", "nC4H10", "iC5H12", "nC5H12", 
                  "nC6H14", "nC7H16", "nC8H18", "nC9H20", "nC10H22", "nC11H24", 
                   "nC12H26", "nC14H30", "N2", "H2O", "CO2", "C15+"]

nwe = [0.9834, 0.0061, 0.0015, 0.0003, 0.0003, 0.00055, 0.0004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003, 0.0, 0.0008, 0.0]

dict_composition= {list_names[i]: nwe[i] for i in range(len(nwe))}

mixture = Mixture(list_of_species, dict_composition)

volumn_desviation = [0] * 19

vlv = valve(kv=0.38)
Vpp = 2.0 
Lc = 2.0 
A1 = 2.6e-3
gas = gc_eos_class(mixture, 300, 4500, None, 1, 0, Aij, volumn_desviation, 'gas')
comp = CompressorClass()
visc = viscosity(mixture, volumn_desviation)
compressor = compression(gas, comp, visc)
plenum_sys = plenum(gas, compressor, vlv, Vpp, Lc, A1)

In [3]:
nAlphas = 300

sampler_N_RotS = qmc.LatinHypercube(d=1)  # d=1 porque estamos amostrando uma única variável
samples_N_RotS = sampler_N_RotS.random(n=nAlphas+1)
N_RotS = qmc.scale(samples_N_RotS, 600, 750).flatten()  # Redimensiona para 1D
sampler_alphas = qmc.LatinHypercube(d=1)
samples_alphas = sampler_alphas.random(n=nAlphas+1)
alphas = qmc.scale(samples_alphas, 0.36, 0.65).flatten()

nData = 40
dt = 2
timestep = 3
x0 = [14.9919, 339.69, 0.42885]
z0 = [6245.39, 6245.39, 321.672, 0.445562, 319.423, 0.503621, 320.097, 0.396345, 339.69, 0.42885, 0.514917]
u0 = [4500, 300, 600, alphas[0], 5000]

In [4]:
# sim = Simulation(plenum_sys, compressor, x0, z0, u0, nAlphas, alphas, N_RotS, nData, dt, timestep)
# x_values, z_values, time_steps, alpha_values, N_values, x_train, y_train, x_min, x_max, y_min,y_max = sim.run()

# sim_data = {
#     "x_values": x_values,
#     "z_values": z_values,
#     "time_steps": time_steps,
#     "x_train": x_train,
#     "y_train": y_train,
#     "x_min": x_min,
#     "x_max": x_max,
#     "y_min": y_min,
#     "y_max": y_max

# }

# np.savez(
#     "simulation_data.npz",
#     x_values=x_values,
#     z_values=z_values,
#     time_steps=time_steps,
#     alpha_values=alpha_values,
#     N_values=N_values,
#     x_train=x_train,
#     y_train=y_train,
#     x_min=x_min,
#     x_max=x_max,
#     y_min = y_min,
#     y_max = y_max

# )


In [5]:
data = np.load("simulation_data.npz")

x_values = data["x_values"]
z_values = data["z_values"]
time_steps = data["time_steps"]
alpha_values = data["alpha_values"]
N_values = data["N_values"]
x_train = data["x_train"]
y_train = data["y_train"]
x_min = data["x_min"]
x_max = data["x_max"]
y_min = data["y_min"]
y_max = data["y_max"]


x_min = torch.tensor(x_min, dtype = torch.float32)
x_max = torch.tensor(x_max, dtype = torch.float32)
y_min = torch.tensor(y_min, dtype = torch.float32)
y_max = torch.tensor(y_max, dtype = torch.float32)
x_train = torch.tensor(np.array(x_train), dtype=torch.float32)
y_train = torch.tensor(np.array(y_train), dtype=torch.float32)
x_train_split, X_val, y_train_split, y_val = train_test_split(
    x_train, 
    y_train, 
    test_size=0.3,  
    random_state=42,  
    shuffle=True      
)
y_train_split = y_train_split.squeeze()
y_val = y_val.squeeze()
train_dataset = torch.utils.data.TensorDataset(x_train_split, y_train_split)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size= 64, shuffle=True)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size= 64, shuffle=False)

In [6]:
model = MyModel(156, dt, x_max, x_min, y_min, y_max, plenum_sys)# Criar a instância do modelo novamente

In [ ]:
train_loss_values = model.train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    lr= 1e-4,
    epochs=4,
    optimizers=torch.optim.Adam,
    patience=100,
    factor=0.5,
    gas = gas
)

# APAGAR DAQUI

In [11]:
 model, train_loader, val_loader, lr, epochs, optimizers, patience, factor, gas = model, train_loader, val_loader, 1e-4, 4, torch.optim.Adam, 100, 0.5, gas

In [22]:
@staticmethod
def _process_gas(args, gas_template):
        y_pred_i, inputs_i, gas_template = args
        gas = gas_template.copy_change_conditions(y_pred_i[1].item(), None, y_pred_i[2].item(), 'gas')
        gas2 = gas_template.copy_change_conditions(y_pred_i[1].item(), y_pred_i[3].item(), None, 'gas')
        gas.evaluate_der_eos_P()
        return gas2.V.item(), gas.dPdV, gas.dPdT
    
def process_gas_batch(y_pred, inputs, gas_template, n_jobs=-1):
        args_list = [(y_pred[i], inputs[i], gas_template) for i in range(y_pred.shape[0])]
        
        with Parallel(n_jobs=n_jobs) as parallel:
            results = parallel(delayed(_process_gas)(args, gas_template) for args in args_list)
        
        Vp, dP_dV, dP_dT = zip(*results)
        return torch.tensor(Vp, dtype=torch.float32), torch.tensor(dP_dV, dtype=torch.float32), torch.tensor(dP_dT, dtype=torch.float32)

In [54]:
def system_residuals(z, x0, u0, plenum_sys):
        """Calcula apenas os resíduos algébricos, mantendo x0 fixo"""
        print(z, x0, u0)
        _, alg_sym = plenum_sys.evaluate_dae(None, x0, z, u0)
        return np.array([alg_sym[i] for i in range(11)])  # Retorna apenas as equações algébricas

def compute_steady_state(u0, plenum_sys, x0, z0):
        """Calcula apenas o estado estacionário das variáveis algébricas"""
        sol = fsolve(system_residuals, z0, args=(x0, u0, plenum_sys))
        return x0, sol  # Retorna x0 (fixo) e z calculado
    
@staticmethod
def _process_steady_state(args, plenum_sys):
        u0, x0, z0 = args
        return compute_steady_state(u0, plenum_sys, x0, z0)
    
def compute_steady_state_batch(u0_batch, plenum_sys, x0_batch, z0_batch, n_jobs=-1):
        process_fn = partial(_process_steady_state, plenum_sys=plenum_sys)
        args_list = list(zip(u0_batch, x0_batch, z0_batch))
        
        with Parallel(n_jobs=n_jobs) as parallel:
            results = parallel(delayed(process_fn)(args) for args in args_list)
        
        x_ss_batch, z_ss_batch = zip(*results)
        return np.stack(x_ss_batch), np.stack(z_ss_batch)

In [49]:
        optimizer = optimizers(model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=factor, patience=patience
        )

        model.train()
        train_loss_values = []
        val_loss_values = []
        physics_loss_values = []
        coeff = torch.tensor([11, -18, 9, -2], dtype=torch.float32) / (6 * dt)
        for epoch in range(epochs):
            model.train()
            total_loss = 0
            total_loss_physics = 0
            
            for batch_idx, (inputs, y_true) in enumerate(train_loader):
                start_time = time.time()
                optimizer.zero_grad()
                
                # Forward Pass
                y_pred = model(inputs)
                
                # Loss de dados
                loss_data = (
                    1e2 * torch.mean((y_true[:, 0] - y_pred[:, 0]) ** 2) +
                    1e1 * torch.mean((y_true[:, 1] - y_pred[:, 1]) ** 2) +
                    1e-6 * torch.mean((y_true[:, 3] - y_pred[:, 3]) ** 2) +
                    1e-6 * torch.mean((y_true[:, 4] - y_pred[:, 4]) ** 2) +
                    1e-4 * torch.mean((y_true[:, 11] - y_pred[:, 11]) ** 2)
                )
                
                # Cálculo das derivadas temporais (vetorizado)
                m_t = torch.sum(coeff.view(1, -1) * torch.cat([
                    y_true[:, 0:1], 
                    inputs[:, -3:, 0]
                ], dim=1), dim=1)
                
                t_t = torch.sum(coeff.view(1, -1) * torch.cat([
                    y_true[:, 1:2], 
                    inputs[:, -3:, 1]
                ], dim=1), dim=1)
                
                P_t = torch.sum(coeff.view(1, -1) * torch.cat([
                    y_true[:, 3:4], 
                    inputs[:, -3:, 2]
                ], dim=1), dim=1)
                
                # Processamento paralelo das propriedades do gás
                Vp, dP_dV, dP_dT = process_gas_batch(y_pred, inputs, gas)
                
                # Cálculo do estado estacionário em paralelo (apenas para z)
                with torch.no_grad():
                    u0_batch = np.stack([
                        np.array([4500, 300, inputs[i, -1, -1].item(), 
                                inputs[i, -1, -2].item(), 5000])
                        for i in range(inputs.shape[0])
                    ])
                    x0_batch = y_pred[:, :3].detach().numpy()
                    z0_batch = y_true[:, 3:].detach().numpy()
                    break
                    # x_ss, z_ss = compute_steady_state_batch(u0_batch, plenum, x0_batch, z0_batch, n_jobs = -1)
                    # z_ss = torch.tensor(z_ss, dtype=torch.float32)


In [50]:
u0_batch, plenum_sys, x0_batch, z0_batch, n_jobs = u0_batch, plenum, x0_batch, z0_batch,  -1

In [ ]:
        process_fn = partial(_process_steady_state, plenum_sys=plenum_sys)
        args_list = list(zip(u0_batch, x0_batch, z0_batch))
        
        with Parallel(n_jobs=n_jobs) as parallel:
            results = parallel(delayed(process_fn)(args) for args in args_list)
        
        x_ss_batch, z_ss_batch = zip(*results)
        # return np.stack(x_ss_batch), np.stack(z_ss_batch)

In [52]:
partial(_process_steady_state, plenum_sys=plenum_sys)

functools.partial(<staticmethod(<function _process_steady_state at 0x71fbf6dc69e0>)>, plenum_sys=<class 'libs.plenum_system.plenum'>)

[4500, 300, 600, 0.624488779189459, 5000]

In [ ]:
compute_steady_state(u0, plenum_sys, x0, z0)

#ATE AQUI

In [ ]:
nAlphas_teste = 7

sampler_N_RotS_teste = qmc.LatinHypercube(d=1)  # d=1 porque estamos amostrando uma única variável
samples_N_RotS_teste = sampler_N_RotS_teste.random(n=nAlphas_teste+1)
N_RotS_teste = qmc.scale(samples_N_RotS_teste, 600, 750).flatten()  # Redimensiona para 1D
sampler_alphas_teste = qmc.LatinHypercube(d=1)
samples_alphas_teste = sampler_alphas_teste.random(n=nAlphas_teste+1)
alphas_teste = qmc.scale(samples_alphas_teste, 0.35, 0.65).flatten()

sim2 = sim = Simulation(plenum_sys, compressor, x0, z0, u0, nAlphas_teste, alphas_teste, N_RotS_teste, 40, dt, timestep)
x_values_teste, z_values_teste, time_steps, alpha_values_teste, N_values_teste, x_teste, y_teste, x_min, x_max, y_min, y_max = sim.run()

In [ ]:
import torch

# Rodar a inferência corretamente
model.eval()  # Importante colocar a rede em modo de avaliação
with torch.no_grad():
    y_pred = model(x_teste)

# Checar se y_pred realmente varia
for name, param in model.named_parameters():
    print(f"{name}: mean={param.mean().item()}, std={param.std().item()}")



In [ ]:
import matplotlib.pyplot as plt

# Remover dimensões extras de y_pred e y_teste (se houver)
y_pred = y_pred.squeeze()
y_teste = y_teste.squeeze()

# Número de variáveis de saída
num_outputs = y_teste.shape[1]

# Ajustar o tamanho da lista time_steps para corresponder aos dados de teste
time_steps = time_steps[:y_teste.shape[0]]
 
# Criar os gráficos separadamente para cada saída
for i in range(num_outputs):
    plt.figure(figsize=(8, 4))  # Criar uma nova figura para cada gráfico
    plt.plot(time_steps, y_teste[:, i], label="Saída Esperada (y_teste)", color="red", linestyle="--")
    plt.plot(time_steps, y_pred[:, i], label="Saída da Rede (y_pred)", color="blue", linestyle="-")
    
    plt.xlabel("Time Step")
    plt.ylabel("Valor")
    plt.title(f"Saída {i}")  # Título do gráfico indicando o índice da saída
    plt.legend()
    plt.grid(True)
    
    plt.show()  # Mostrar o gráfico

"[0, 1, 2, 3, 5, 7, 9, 11]"